## GSE33980 - oxidative stress response
### Reproducing paper results

#### Co-cluster genes

In [1]:
import pandas as pd
from IPython.display import display_html

def cocluster(df_one_grouped, df_two_grouped):
    co_cluster_dict = dict()
    for one_name, one_group in df_one_grouped:
        co_cluster_dict[one_name] = dict()
        for two_name, two_group in df_two_grouped:
            co_cluster_dict[one_name][two_name] = one_group["gene"].isin(two_group["gene"]).sum()

    return pd.DataFrame(co_cluster_dict)

def show_clusters_matrices(df_cocluster, cr_label="Column vs. Row highlight", rc_label="Row vs. Column highLight"):
    col_vs_row = df_cocluster.style.background_gradient(
        axis=0, 
        cmap="cividis",
    ).set_table_attributes("style='display:inline'").set_caption(cr_label)

    row_vs_col = df_cocluster.style.background_gradient(
        axis=1, 
        cmap="cividis",
    ).set_table_attributes("style='display:inline'").set_caption(rc_label)
    
    space = "\xa0" * 10

    return display_html(
        col_vs_row._repr_html_() + space + row_vs_col._repr_html_(),
        raw=True
    )

In [2]:
optimal_clusters_control = pd.read_csv("DPGP/results/GSE33980/paper/geo_ura3_optimal_clustering.txt", sep="\t")
optimal_clusters_mutant = pd.read_csv("DPGP/results/GSE33980/paper/geo_d258_optimal_clustering.txt", sep="\t")

optimal_clusters_control_grouped = optimal_clusters_control.groupby("cluster")
optimal_clusters_mutant_grouped = optimal_clusters_mutant.groupby("cluster")

control_vs_mutant = cocluster(optimal_clusters_mutant_grouped, optimal_clusters_control_grouped)
show_clusters_matrices(control_vs_mutant, cr_label="Mutant vs. Control highlight", rc_label="Control vs. Mutant highlight")

,1,2,3,4,5,6
1,30,44,7,1,1,0
2,11,64,12,2,3,1
3,0,3,56,8,18,0
4,11,10,125,3,29,0
5,17,35,78,8,16,1
6,0,1,14,2,5,0
,1,2,3,4,5,6
1,30,44,7,1,1,0
2,11,64,12,2,3,1
3,0,3,56,8,18,0


#### Equivalent clusters: Control vs. Mutant

In [4]:
from scipy.stats import pearsonr

mean_expr_ura3 = pd.read_csv("DPGP/results/GSE33980/paper/geo_ura3_mean_expression.csv")
mean_expr_d258 = pd.read_csv("DPGP/results/GSE33980/paper/geo_d258_mean_expression.csv")

mat_corr = list()
for ctr_cluster in mean_expr_ura3.columns:
    temp = list()
    x = mean_expr_ura3[ctr_cluster]
    for mut_cluster in mean_expr_d258.columns:
        y = mean_expr_d258[mut_cluster]
        r, p = pearsonr(x, y)
        temp.append((r, p))
    mat_corr.append(temp)

# mat_corr = pd.DataFrame(mat_corr)

mat_corr_r = list()
mat_corr_p = list()
for i in mat_corr:
    temp_r = list()
    temp_p = list()
    for j in i:
        temp_r.append(j[0])
        temp_p.append(j[1])
    mat_corr_r.append(temp_r)
    mat_corr_p.append(temp_p)

mat_corr_r = pd.DataFrame(mat_corr_r)
mat_corr_p = pd.DataFrame(mat_corr_p)

col_labels = dict()
for i in mat_corr_r.columns:
    col_labels[i] = f"mut cluster {i+1}"

idx_labels = dict()
for i in mat_corr_r.index:
    idx_labels[i] = f"ctrl cluster {i+1}"

mat_corr_r.rename(columns=col_labels, index=idx_labels, inplace=True)
mat_corr_p.rename(columns=col_labels, index=idx_labels, inplace=True)

mat_corr_r

,mut cluster 1,mut cluster 2,mut cluster 3,mut cluster 4,mut cluster 5,mut cluster 6
ctrl cluster 1,0.886733,-0.008017,0.112800,-0.331487,-0.152816,-0.643181
ctrl cluster 2,0.259894,0.770829,-0.766273,0.260113,-0.711813,-0.316136
ctrl cluster 3,-0.712250,-0.645733,0.570376,0.165090,0.709746,0.470446
ctrl cluster 4,0.086582,-0.913142,0.933808,-0.177311,0.810437,-0.151423
ctrl cluster 5,-0.471781,0.793871,-0.834994,0.044242,-0.685248,0.668208
ctrl cluster 6,-0.828019,0.077701,-0.153755,0.004691,0.039778,0.891481


> Gene trajectories for each of the control and ΔrosR strains were clustered in
independent DPGP modeling runs. Resultant clusters were analyzed to determine how each
gene changed cluster membership in response to the rosR mutation. We computed the Pearson
correlation coefficient in mean trajectory between all control clusters and all ΔrosR clusters.
Clusters with the highest coefficients across conditions were considered equivalent
across strains.

In [5]:
sel_row = mat_corr_r.idxmax(axis=1)

for i in sel_row.index:
    print(
        i, " -> ", sel_row[i], 
        f" (r={mat_corr_r.loc[i, sel_row[i]]:.4}, p={mat_corr_p.loc[i, sel_row[i]]:.4})"
    )

ctrl cluster 1  ->  mut cluster 1  (r=0.8867, p=5.353e-169)
ctrl cluster 2  ->  mut cluster 2  (r=0.7708, p=1.385e-99)
ctrl cluster 3  ->  mut cluster 5  (r=0.7097, p=8.569e-78)
ctrl cluster 4  ->  mut cluster 3  (r=0.9338, p=1.908e-224)
ctrl cluster 5  ->  mut cluster 2  (r=0.7939, p=1.168e-109)
ctrl cluster 6  ->  mut cluster 6  (r=0.8915, p=2.33e-173)


In [6]:
sel_col = mat_corr_r.idxmax(axis=0)

for i in sel_col.index:
    print(
        i, " -> ", sel_col[i], 
        f" (r={mat_corr_r.loc[sel_col[i], i]:.4}, p={mat_corr_p.loc[sel_col[i], i]:.4})"
    )

mut cluster 1  ->  ctrl cluster 1  (r=0.8867, p=5.353e-169)
mut cluster 2  ->  ctrl cluster 5  (r=0.7939, p=1.168e-109)
mut cluster 3  ->  ctrl cluster 4  (r=0.9338, p=1.908e-224)
mut cluster 4  ->  ctrl cluster 2  (r=0.2601, p=3.555e-09)
mut cluster 5  ->  ctrl cluster 4  (r=0.8104, p=9.83e-118)
mut cluster 6  ->  ctrl cluster 6  (r=0.8915, p=2.33e-173)


| Control strain clusters | Mutant strain clusters |
| :---: | :---: |
|<img src="DPGP/results/GSE33980/paper/geo_ura3_gene_expression_fig_1.png" width="500">|<img src="DPGP/results/GSE33980/paper/geo_d258_gene_expression_fig_1.png" width="500">|

In [7]:
control_vs_mutant_new = control_vs_mutant.rename(
    columns={i: f"mut cluster {i}" for i in control_vs_mutant.columns}, 
    index={i: f"ctrl cluster {i}" for i in control_vs_mutant.index}
)

# sel_new = [(b, a) for a, b in sel_col.items()]
sel_new = [(a, b) for a, b in sel_row.items()]

for idx, col in sel_new:
    print(idx, " -> ", col, f"\tn genes in both clusters: {control_vs_mutant_new.loc[idx, col]:3}")

ctrl cluster 1  ->  mut cluster 1 	n genes in both clusters:  30
ctrl cluster 2  ->  mut cluster 2 	n genes in both clusters:  64
ctrl cluster 3  ->  mut cluster 5 	n genes in both clusters:  18
ctrl cluster 4  ->  mut cluster 3 	n genes in both clusters: 125
ctrl cluster 5  ->  mut cluster 2 	n genes in both clusters:  35
ctrl cluster 6  ->  mut cluster 6 	n genes in both clusters:   0


In [8]:
df = pd.DataFrame(
    "", 
    index=[f"ctrl cluster {i}" for i in range(1,7)],
    columns=[f"mut cluster {i}" for i in range(1,6)],
)
for row, col in sel_new:
    df.loc[row, col] = 'background-color: yellow;color: black;'

control_vs_mutant_new.style.apply(lambda x: df, axis=None)

,mut cluster 1,mut cluster 2,mut cluster 3,mut cluster 4,mut cluster 5,mut cluster 6
ctrl cluster 1,30,44,7,1,1,0
ctrl cluster 2,11,64,12,2,3,1
ctrl cluster 3,0,3,56,8,18,0
ctrl cluster 4,11,10,125,3,29,0
ctrl cluster 5,17,35,78,8,16,1
ctrl cluster 6,0,1,14,2,5,0


#### Significance of overrepresentation in cluster switching

In [9]:
import pandas as pd
from scipy.stats import fisher_exact


def get_contingency_tables(df_cocluster: pd.DataFrame, corresponding_cluster: list[tuple]) -> dict[tuple,list]:
    n = df_cocluster.values.sum()
    contingency_dict = dict()

    for ctrl_cluster, mut_cluster in corresponding_cluster:
        ctrl = df_cocluster.loc[ctrl_cluster, :]
        mut = df_cocluster.loc[:, mut_cluster]
        in_ctrl_in_mut = ctrl.pop(mut_cluster)
        in_ctrl_no_mut = ctrl.sum()
        no_ctrl_in_mut = mut.drop(ctrl_cluster).sum()
        no_ctrl_no_mut = n - (in_ctrl_in_mut + in_ctrl_no_mut + no_ctrl_in_mut)

        contingency_dict[(ctrl_cluster, mut_cluster)] = [
            [in_ctrl_in_mut, in_ctrl_no_mut],
            [no_ctrl_in_mut, no_ctrl_no_mut]
        ]
    
    return contingency_dict

def get_clusters_fishers_exact_test(contingency_dict: dict[tuple,list]) -> dict[tuple,tuple]:
    clusters_fet = dict()

    for corresponding_clusters, cont_table in contingency_dict.items():
        res = fisher_exact(cont_table)
        clusters_fet[corresponding_clusters] = res

    return clusters_fet


Contingency tables of number of genes:
| | Belong to mutant cluster *n* | Do NOT belong to mutant cluster *n* |
| :---: | :---: | :---: |
| **Belong to control cluster *m*** | $x$ | $y=row - x$ |
| **Do NOT belong to control cluster *m*** | $z=col - x$ | $616 - (x+y+z)$ |

where
- $x$ is the number of genes belonging to both clusters
- $y$ is the number of remaining genes in the control cluster
- $z$ is the number of remaining genes in the mutant cluster
- the last cell is the number of genes not belonging to any of those two clusters

In [10]:
contingency_tables = get_contingency_tables(control_vs_mutant_new, sel_new)
contingency_tables

{('ctrl cluster 1', 'mut cluster 1'): [[30, 53], [39, 494]],
 ('ctrl cluster 2', 'mut cluster 2'): [[64, 29], [93, 430]],
 ('ctrl cluster 3', 'mut cluster 5'): [[18, 67], [54, 477]],
 ('ctrl cluster 4', 'mut cluster 3'): [[125, 53], [167, 271]],
 ('ctrl cluster 5', 'mut cluster 2'): [[35, 120], [122, 339]],
 ('ctrl cluster 6', 'mut cluster 6'): [[0, 22], [2, 592]]}

In [11]:
fet = get_clusters_fishers_exact_test(contingency_tables)
fet

{('ctrl cluster 1', 'mut cluster 1'): (7.169811320754717,
  3.080963815917701e-11),
 ('ctrl cluster 2', 'mut cluster 2'): (10.203930292918058,
  4.090025389644097e-22),
 ('ctrl cluster 3', 'mut cluster 5'): (2.373134328358209,
  0.005919611703544562),
 ('ctrl cluster 4', 'mut cluster 3'): (3.827251158061236,
  5.305365754543032e-13),
 ('ctrl cluster 5', 'mut cluster 2'): (0.8104508196721312,
  0.39411554547755806),
 ('ctrl cluster 6', 'mut cluster 6'): (0.0, 1.0)}

In [12]:
tot_diff_gene = int()

for idx, col in sorted(sel_new, key=lambda x: x[0]):
    pearson_r = mat_corr_r.loc[idx, col]
    pearson_p = mat_corr_p.loc[idx, col]
    fishers_p = fet[(idx, col)][1]
    diff_gene = contingency_tables[(idx, col)][0][1]
    tot_diff_gene += diff_gene
    print(
        idx, " -> ", col, "\t",
        f"(r={pearson_r:.4}, p={pearson_p:.2e})\n",
        "\t" * 5,
        f"Fisher's exact test: p={fishers_p:.2e}",
        "-> significant relationship\n" if fishers_p < 0.05 else "\n",
        "\t" * 5,
        f"genes with different dynamic: {diff_gene:3}"
    )

print(f"Total number of genes with different dynamic: {tot_diff_gene}")

# cluster 5
print("\nCluster 5: upregulation after 40'")
tot_cluster5_genes = control_vs_mutant_new.loc['ctrl cluster 5', :].sum()
inv_dynamic_genes = control_vs_mutant_new.loc["ctrl cluster 5",:][mat_corr_r.loc["ctrl cluster 5",:] < -0.5]
tot_inv_dynamic_genes = inv_dynamic_genes.sum()

print(
    "Total number of genes in control cluster:".rjust(63),
    f"{tot_cluster5_genes:4}\n",
    "Genes with inverted dynamic in corresponding mutant clusters:".rjust(62),
    f"{tot_inv_dynamic_genes:4}", f"({tot_inv_dynamic_genes/tot_cluster5_genes:.2%})"
)
for i, (cluster, n) in enumerate(inv_dynamic_genes.items()):
    txt = "of which" if i==0 else ""
    print(
        txt.rjust(62),
        f"{n:5} in {cluster}",
    )

print(
    "Paper results", "\n"
    "Total number of genes with different dynamic: 372", "\n"
    "Cluster upregulated after 40'", "\n",
    "Total number of genes in control cluster:  141".rjust(67), "\n",
    "Genes with inverted dynamic in corresponding mutant clusters:   89".rjust(67), f"({89/141:.2%})", "\n",
    "of which   72".rjust(67), "in mutant cluster 3", "\n",
    "17".rjust(67), "in mutant cluster 5"
)


ctrl cluster 1  ->  mut cluster 1 	 (r=0.8867, p=5.35e-169)
 					 Fisher's exact test: p=3.08e-11 -> significant relationship
 					 genes with different dynamic:  53
ctrl cluster 2  ->  mut cluster 2 	 (r=0.7708, p=1.38e-99)
 					 Fisher's exact test: p=4.09e-22 -> significant relationship
 					 genes with different dynamic:  29
ctrl cluster 3  ->  mut cluster 5 	 (r=0.7097, p=8.57e-78)
 					 Fisher's exact test: p=5.92e-03 -> significant relationship
 					 genes with different dynamic:  67
ctrl cluster 4  ->  mut cluster 3 	 (r=0.9338, p=1.91e-224)
 					 Fisher's exact test: p=5.31e-13 -> significant relationship
 					 genes with different dynamic:  53
ctrl cluster 5  ->  mut cluster 2 	 (r=0.7939, p=1.17e-109)
 					 Fisher's exact test: p=3.94e-01 
 					 genes with different dynamic: 120
ctrl cluster 6  ->  mut cluster 6 	 (r=0.8915, p=2.33e-173)
 					 Fisher's exact test: p=1.00e+00 
 					 genes with different dynamic:  22
Total number of genes with different dynamic: 34